In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [105]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

parking_dataset=pd.read_csv('parking_dataset.csv')
parking_dataset['Timestamp'] = pd.to_datetime(parking_dataset['Timestamp'], format='%d-%m-%Y')

# Preprocessing
parking_dataset['ParkingSpaceID'].fillna(parking_dataset['ParkingSpaceID'].mean(), inplace=True)
parking_dataset['OccupancyStatus'].fillna(parking_dataset['OccupancyStatus'].mode()[0], inplace=True)
parking_dataset['VehicleType'].fillna(parking_dataset['VehicleType'].mode()[0], inplace=True)

# Encode categorical variables
label_encoder = LabelEncoder()
parking_dataset['VehicleType'] = label_encoder.fit_transform(parking_dataset['VehicleType'])
parking_dataset['OccupancyStatus'] = label_encoder.fit_transform(parking_dataset['OccupancyStatus'])

# SARIMA modeling
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 7)
sarima_model = SARIMAX(parking_dataset['OccupancyStatus'], order=order, seasonal_order=seasonal_order)
results = sarima_model.fit(disp=False)

# Streamlit App
st.title("Smart Parking System SARIMA Model")
st.subheader('Data:')
st.write(parking_dataset)

# User Input for Predicting Parking Status
st.sidebar.header("Predict Parking Status:")
parking_space_id_input = st.sidebar.number_input("Enter Parking Space ID:", format="%d")
prediction_button = st.sidebar.button("Predict")

# Predict and Display Result
if prediction_button:
    user_input_df = pd.DataFrame({'ParkingSpaceID': [parking_space_id_input]})
    prediction = int(results.predict(start=len(parking_dataset), end=len(parking_dataset) + 1, dynamic=False).iloc[0])
    prediction_label = label_encoder.inverse_transform([prediction])[0]
    st.sidebar.subheader("Prediction:")
    st.sidebar.write(f"Parking Space {parking_space_id_input} is {prediction_label}")


    # Plot SARIMA Forecast
    st.subheader("SARIMA Model Forecast:")
    forecast_periods = 7
    forecast_index = pd.date_range(start=parking_dataset['Timestamp'].iloc[-1] + pd.DateOffset(days=1), periods=forecast_periods, freq='D')

    # Generate the forecast
    forecast = results.get_forecast(steps=forecast_periods)
    forecast_values = forecast.predicted_mean.values

    # Plotting
    fig, ax = plt.subplots()
    ax.plot(parking_dataset['Timestamp'], parking_dataset['OccupancyStatus'], label='Observed Data')  # Plot observed data
    ax.plot(forecast_index, forecast_values, 'g--', label='SARIMA Forecast')  # Plot SARIMA forecast
    ax.legend()

    # Display the plot in Streamlit
    st.pyplot(fig)



Overwriting app.py


In [106]:
!wget -q -O - ipv4.icanhazip.com

35.229.19.170


In [107]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] / fetchMetadata: sill resolveWithNewModule ms@2.1.2 checki


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.19.170:8501

npx: installed 22 in 2.539s
your url is: https://two-rings-wish.loca.lt
  Stopping...
^C
